In [23]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD, Adam

In [2]:
FAST_RUN = False
IMAGE_WIDTH=256
IMAGE_HEIGHT=256
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
# Define a list of possible batch sizes
BATCH_SIZES = [16, 32, 64, 128]
# Generate a random index to select a batch size from the list
random_index = np.random.randint(0, len(BATCH_SIZES))

# Select a batch size using the random index
BATCH_SIZE = BATCH_SIZES[random_index]
NUM_CLASSES = 1

In [3]:
# Loop through all image files in the directory
def resize_images(path_to_directory, target_size):
  for filename in os.listdir(path_to_directory):
        root, ext = os.path.splitext(filename)
        if ext.lower() in [".jpg", ".jpeg", ".png"]:
            img_path = os.path.join(path_to_directory, filename)
            if Image.open(img_path).size != target_size:
              img = Image.open(img_path).resize(target_size)
              img.save(img_path)
  print(f"All images in {path_to_directory} resized to {target_size}")

In [4]:
# Load the dataset
dataset_path = 'data/landscape/train'
images = []
labels = []

for label in os.listdir(dataset_path):
    if label.startswith('.'):
        continue
    
    label_path = os.path.join(dataset_path, label)
    resize_images(label_path, IMAGE_SIZE)
    for img_path in os.listdir(label_path):
        if os.path.exists(os.path.join(label_path, img_path)):
            if img_path.startswith('.'):
                continue
            img = Image.open(os.path.join(label_path, img_path)).resize(IMAGE_SIZE)
            x = img_to_array(img)
            x /= 255.0  # Normalize pixel values
            img = np.expand_dims(x, axis=0)
            images.append(img)
            labels.append(label)

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

All images in data/landscape/train/forest resized to (256, 256)
All images in data/landscape/train/buildings resized to (256, 256)
All images in data/landscape/train/glacier resized to (256, 256)
All images in data/landscape/train/street resized to (256, 256)
All images in data/landscape/train/mountain resized to (256, 256)
All images in data/landscape/train/sea resized to (256, 256)


In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

class AlexNet():
    def __init__(self, input_shape=(224,224,3), activation='sigmoid', class_size=1):
        super().__init__()
        self.model = Sequential()

        # 1st Convolutional Layer
        self.model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=input_shape))
        self.model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

        # 2nd Convolutional Layer
        self.model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
        self.model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

        # 3rd Convolutional Layer
        self.model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))

        # 4th Convolutional Layer
        self.model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))

        # 5th Convolutional Layer
        self.model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
        self.model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

        # Flatten the layer
        self.model.add(Flatten())

        # 1st Dense Layer
        self.model.add(Dense(4096, activation='relu'))
        self.model.add(Dropout(0.5))

        # 2nd Dense Layer
        self.model.add(Dense(4096, activation='relu'))
        self.model.add(Dropout(0.5))

        # 3rd Dense Layer
        self.model.add(Dense(class_size, activation=activation))


    def load_weights(self, filepath):
        self.model.load_weights(filepath)
        
        
    def save_weights(self, filepath):
        self.model.save_weights(filepath)


    def compile(self, loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']):
        # Compile the model
        self.model.compile(loss=loss, optimizer=optimizer, metrics=metrics)



In [11]:
# !pip install datasets

# from datasets import load_dataset
# import datasets

# !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
# !unzip -q tiny-imagenet-200.zip -d tiny-imagenet

# imagenet = datasets.load_dataset(
#     'tiny_imagenet',
#     split='validation',
#     ignore_verifications=True  # set to True if seeing splits Error
# )
# imagenet

In [17]:
# Set up the image data generator
datagen = ImageDataGenerator(rescale=1./255)

# Set up the directory paths for the dataset
train_dir = 'data/tiny-imagenet-200/train'
val_dir = 'data/tiny-imagenet-200/val'

# Load the training set using the ImageDataGenerator
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)

# Load the validation set using the ImageDataGenerator
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

# Print the class labels
print(val_generator.class_indices)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 1 classes.
{'images': 0}


In [ ]:
# Create an instance of your AlexNet model
alexnet = AlexNet(input_shape=(224, 224, 3), class_size=NUM_CLASSES)

# Compile the model with appropriate optimizer and loss function
alexnet.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training set
history = alexnet.model.fit(train_generator, epochs=10, validation_data=val_generator)

# Save the model weights to a file
# alexnet.save_weights('alexnet_weights.h5')


Epoch 1/10
1248/1563 [======================>.......] - ETA: 8:05 - loss: 0.0329 - accuracy: 0.9945

In [12]:

from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50


# Load the models
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inception_v3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
alexnet = AlexNet(input_shape=(224, 224, 3))
alexnet = alexnet.load_weights()


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for imagenet

In [ ]:
from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Add your custom layers on top of the base model
model = Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model and specify the loss function, optimizer, and evaluation metric(s)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create data generator with data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create data generator without data augmentation
val_datagen = ImageDataGenerator()

# Define batch size
batch_size = 32

# Create training data generator
train_generator = train_datagen.flow(
    train_images,
    train_labels,
    batch_size=batch_size
)

# Create validation data generator
val_generator = val_datagen.flow(
    val_images,
    val_labels,
    batch_size=batch_size
)



In [ ]:
# Compile the models
vgg16.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
inception_v3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
resnet50.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
alexnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the models
vgg16.fit(train_generator, epochs=10, validation_data=val_generator)
inception_v3.fit(train_generator, epochs=10, validation_data=val_generator)
resnet50.fit(train_generator, epochs=10, validation_data=val_generator)
alexnet.fit(train_generator, epochs=10, validation_data=val_generator)

In [ ]:
# Evaluate the models
vgg16.evaluate(val_generator)
inception_v3.evaluate(val_generator)
resnet50.evaluate(val_generator)
alexnet.evaluate(val_generator)


In [ ]:
# Load an image
img = cv2.imread('/path/to/image')

# Preprocess the image
img = cv2.resize(img, (224, 224))
img = img.astype('float32') / 255.0

# Make a prediction using the models
vgg16_pred = vgg16.predict(np.array([img]))
inception_v3_pred = inception_v3.predict(np.array([img]))
resnet50_pred = resnet50.predict(np.array([img]))
alexnet_pred = alexnet.predict(np.array([img]))

# Print the predictions
print('VGG16:', vgg16_pred)
print('InceptionV3:', inception_v3_pred)
print('Resnet50:', resnet50_pred)
print('AlexNet:', alexnet_pred)
